## Load the DataBricks table as Spark DataFrame

In [2]:
df = spark.table("policedata")

df = df.drop('FallsWithin')
df = df.drop('Lat')
df = df.drop('Long')
df = df.drop('Context')
df = df.drop('Location')
df = df.drop('LSOACode')
df = df.drop('LSOAName')
df = df.drop('CrimeID')

df = df.na.drop(subset=["Outcome"])
df = df.na.drop(subset=["ReportedBy"])
df = df.na.drop(subset=["Month"])

# Filter subset (remove when doing full training)
df.registerTempTable("temp")
df = sqlContext.sql("select * from temp where Month = '2017-01'")

print("Working with", "{:,}".format(df.count()), "rows")

Working with 353,837 rows

## Encoding and feature extraction

In [4]:
from pyspark.ml.feature import StringIndexer

# Encode the Outcome for our mapping  
indexer = StringIndexer(inputCol="Outcome", outputCol="Outcome_e")
indexerModel = indexer.fit(df)
OutcomeMap = {}
for idx, lab in enumerate(indexerModel.labels):
  OutcomeMap[lab] = idx   
df = indexerModel.transform(df)

# !IMPORTANT! This maps outcomes to our caught/not-caught for the crime: 1 = caught, 0 = not caught
#print(OutcomeMap)
mapOutcomeUDF = udf(lambda x: 1 if x in [6, 15, 18, 7, 12, 22, 24, 21, 20, 17, 13] else 0, IntegerType())
df = df.withColumn("y", mapOutcomeUDF(df.Outcome_e))

# Encode the Crime and ReportedBy columns we'll be training on 
indexer = StringIndexer(inputCol="ReportedBy", outputCol="ReportedBy_e")
indexerModel = indexer.fit(df)
ReportedByMap = {}
for idx, lab in enumerate(indexerModel.labels):
  ReportedByMap[lab] = idx    
df = indexerModel.transform(df)

indexer = StringIndexer(inputCol="Crime", outputCol="Crime_e")
indexerModel = indexer.fit(df)
CrimeMap = {}
for idx, lab in enumerate(indexerModel.labels):
  CrimeMap[lab] = idx      
df = indexerModel.transform(df)  

# Finally add the month as an integer
mapMonthUDF = udf(lambda month: int(month[-2:]), IntegerType())
df = df.withColumn("Month_e", mapMonthUDF(df.Month))

## Convert DF to Numpy array

In [6]:
import numpy as np
training_array = np.array(df.select("y", "ReportedBy_e", "Crime_e", "Month_e").collect())

## Training the model with Scikit

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

# Use RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100)
#model = BaggingClassifier()
X = training_array[0:, 1:].astype(int)
y = training_array[0:, 0].astype(int)
model = model.fit(X, y)

## Test the model

In [10]:
print(CrimeMap)
print()
# Test all forces, with crime 12 = possesion of a weapon
for f, fi in ReportedByMap.items():
    a = model.predict_proba([[fi, 12, 1]])[0]
    print(f, a)

{'Possession of weapons': 12, 'Theft from the person': 9, 'Burglary': 3, 'Other theft': 2, 'Drugs': 7, 'Robbery': 11, 'Violence and sexual offences': 0, 'Vehicle crime': 4, 'Criminal damage and arson': 1, 'Public order': 6, 'Other crime': 8, 'Bicycle theft': 10, 'Shoplifting': 5}

Thames Valley Police [ 0.80341123 0.19658877]
Northumbria Police [ 0.74393532 0.25606468]
Dyfed-Powys Police [ 0.63972469 0.36027531]
North Wales Police [ 0.76714786 0.23285214]
Leicestershire Police [ 0.76868943 0.23131057]
West Mercia Police [ 0.70106202 0.29893798]
Lincolnshire Police [ 0.64432676 0.35567324]
Durham Constabulary [ 0.52467249 0.47532751]
South Yorkshire Police [ 0.86222907 0.13777093]
Cambridgeshire Constabulary [ 0.96678916 0.03321084]
City of London Police [ 0.46047619 0.53952381]
Metropolitan Police Service [ 0.7187991 0.2812009]
Norfolk Constabulary [ 0.74172757 0.25827243]
Lancashire Constabulary [ 0.88625223 0.11374777]
Greater Manchester Police [ 0.83939912 0.16060088]
West Midlands Police [ 0.86940547 0.13059453]
Kent Police [ 0.53960189 0.46039811]
Essex Police [ 0.87165128 0.12834872]
Cleveland Police [ 0.75473883 0.24526117]
Hampshire Constabulary [ 0.83316744 0.16683256]
Humberside Police [ 0.74632645 0.25367355]
North Yorkshire Police [ 0.62191764 0.37808236]
Staffordshire Police [ 0.78379292 0.21620708]
Dorset Police [ 0.75753059 0.24246941]
Hertfordshire Constabulary [ 0.9285687 0.0714313]
Wiltshire Police [ 0.93365162 0.06634838]
Suffolk Constabulary [ 0.79232919 0.20767081]
Merseyside Police [ 0.66320493 0.33679507]
Warwickshire Police [ 0.73016243 0.26983757]
Gloucestershire Constabulary [ 1. 0.]
Bedfordshire Police [ 0.65252909 0.34747091]
South Wales Police [ 0.56263885 0.43736115]
Cheshire Constabulary [ 0.56587477 0.43412523]
Cumbria Constabulary [ 0.86349938 0.13650062]
Nottinghamshire Police [ 0.85505028 0.14494972]
Gwent Police [ 0.80643192 0.19356808]
Surrey Police [ 0.69872882 0.30127118]
Sussex Police [ 0.85101676 0.14898324]
Avon and Somerset Constabulary [ 0.82463743 0.17536257]
West Yorkshire Police [ 0.71799799 0.28200201]
Derbyshire Constabulary [ 0.91265157 0.08734843]
Devon & Cornwall Police [ 0.68436979 0.31563021]
Northamptonshire Police [ 0.97224041 0.02775959]

## Pickle model and push to Azure storage

In [12]:
#
# Widgets are how we get values passed from a DataBricks job
#
try:
  #dbutils.widgets.text("model_version", "1.0.0")
  #dbutils.widgets.text("storage_account", "modelreg")
  #dbutils.widgets.text("model_name", "batcomputer")

  # Model version, name & storage-account is passed into job, and storage key is kept in Azure Key Vault
  STORAGE_KEY       = dbutils.secrets.get("ai-deploy-secrets", "storage-key")
  STORAGE_ACCOUNT   = dbutils.widgets.get("storage_account")
  MODEL_VERSION     = dbutils.widgets.get("model_version")
  STORAGE_CONTAINER = dbutils.widgets.get("model_name")
except:
    pass
    
#
# STORAGE_ACCOUNT value should only be set when this Notebook is invoked via a job
# So we only pickle and store in Azure blobs when running as a job
#
if 'STORAGE_ACCOUNT' in vars():
  print("Saving pickles to:", STORAGE_ACCOUNT, " / ", STORAGE_CONTAINER)
  
  # Create pickles and data lookup 
  from collections import OrderedDict
  import pickle

  lookup = OrderedDict()

  # ORDER IS IMPORTANT! This is why we use OrderedDict and create entries one by one
  lookup["Force"] = FallsWithin_mapping
  lookup["Crime"] = CrimeType_mapping

  # Create output lookup
  flags = ["getaway_proba", "busted_proba"]

  # Pickle the whole damn lot
  with open("model.pkl" , 'wb') as file:  
    pickle.dump(model, file)
    file.close()

  with open("lookup.pkl" , 'wb') as file:  
    pickle.dump(lookup, file)
    file.close()

  with open("flags.pkl" , 'wb') as file:  
    pickle.dump(flags, file)    
    file.close()

  from azure.storage.blob import BlockBlobService

  # Create the BlockBlockService that is used to call the Blob service for the storage account
  block_blob_service = BlockBlobService(account_name=STORAGE_ACCOUNT, account_key=STORAGE_KEY) 

  # Create a container
  block_blob_service.create_container(STORAGE_CONTAINER) 

  # Upload the created file, use local_file_name for the blob name
  block_blob_service.create_blob_from_path(STORAGE_CONTAINER, MODEL_VERSION + "/model.pkl", "model.pkl")
  block_blob_service.create_blob_from_path(STORAGE_CONTAINER, MODEL_VERSION + "/lookup.pkl", "lookup.pkl")
  block_blob_service.create_blob_from_path(STORAGE_CONTAINER, MODEL_VERSION + "/flags.pkl", "flags.pkl")